# Libraries

In [1]:
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [42]:
# Driver's path
path = 'C:/Users/papan/Downloads/chromedriver_win32'
driver = webdriver.Chrome(path)
# Maximize Window
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)
### Login to dummy linked in account and do the same process because of restriction with authwall login

# Enter to the site
driver.get('https://www.linkedin.com/login')
time.sleep(2)
# # Accept cookies
driver.find_element('xpath','//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]').click()
# # User Credentials
# Reading txt file where we have our user credentials
with open('user_credentials.txt', 'r',encoding="utf-8") as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]
user_name = user_credentials[0] # First line
password = user_credentials[1] # Second line
driver.find_element('xpath','//*[@id="username"]').send_keys(user_name)
driver.find_element('xpath','//*[@id="password"]').send_keys(password)
time.sleep(1)
# Login button
driver.find_element('xpath','//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(30)
# if driver.find_element('xpath','//*[@id="ember7"]/h2').text == 'Confirm your account information':
#     driver.find_element('xpath','//*[@id="ember20"]/button[1]').click()
driver.get('https://www.linkedin.com/jobs/search/?currentJobId=3592708886&geoId=104677530&location=Greece')
time.sleep(5)
# driver.execute_script("return window.stop();")
page=1
links = []
print('Links are being collected...')
while True:
    
    try:
        jobs_block = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')
        jobs_list = jobs_block.find_elements(By.TAG_NAME,'li')
        print(f'Collecting links in page {page}')
        for job in jobs_list:
            if job == jobs_list[-1]:
                all_links = BeautifulSoup(job.get_attribute('innerHTML'),'html').find_all('a',href=True)
                for a in all_links:
                    if a['href'].startswith("/jobs/view") and a['href'] not in links: 
                        links.append(a['href'])
                    else:
                        pass
                page += 1
                driver.find_element('xpath',f'//button[@aria-label="Page {page}"]').click()
                time.sleep(3)
                
            else:
                # scroll down for each job element
                driver.execute_script('arguments[0].scrollIntoView();', job)
                all_links = BeautifulSoup(job.get_attribute('innerHTML'),'html').find_all('a',href=True)
                for a in all_links:
                    if a['href'].startswith("/jobs/view") and a['href'] not in links: 
                        links.append(a['href'])
                    else:
                        pass
    except:
        break

C:\Users\papan\AppData\Local\Temp\ipykernel_14616\3848064833.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Links are being collected...


In [133]:
links2 = links[3:5]

In [137]:
# Driver's path
path = 'C:/Users/papan/Downloads/chromedriver_win32'
driver = webdriver.Chrome(path)
# Maximize Window
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)
### Login to dummy linked in account and do the same process because of restriction with authwall login

# Enter to the site
driver.get('https://www.linkedin.com/login')
time.sleep(2)
# # Accept cookies
driver.find_element('xpath','//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]').click()
# # User Credentials
# Reading txt file where we have our user credentials
with open('user_credentials.txt', 'r',encoding="utf-8") as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]
user_name = user_credentials[0] # First line
password = user_credentials[1] # Second line
driver.find_element('xpath','//*[@id="username"]').send_keys(user_name)
driver.find_element('xpath','//*[@id="password"]').send_keys(password)
time.sleep(1)
# Login button
driver.find_element('xpath','//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(30)
# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = [] 
job_desc = []


# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')
for i in range(len(links2)):
    try:
        driver.get('https://www.linkedin.com'+links2[i])
        time.sleep(2)
        # Click See more.
        buttons = driver.find_elements(By.CLASS_NAME,"artdeco-card__actions")
        for button in buttons:
            if button.text == 'See more':
                button.click()
        time.sleep(2)
    except:
        pass
    
    # Find the general information of the job offer
    # Used try, except in case there is some missing information for some of the job offers
    contents = driver.find_elements(By.CLASS_NAME, 'p5')
    for content in contents:
        try:
            job_titles.append(content.find_element(By.TAG_NAME, "h1").text)
        except:
            job_titles.append('')  # Append an empty string if element is not found

        try:
            company_names.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__company-name").text)
        except:
            company_names.append('')  # Append an empty string if element is not found

        try:
            company_locations.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__bullet").text)
        except:
            company_locations.append('')  # Append an empty string if element is not found

        try:
            work_methods.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__workplace-type").text)
        except:
            work_methods.append('')  # Append an empty string if element is not found

        try:
            post_dates.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__posted-date").text)
        except:
            post_dates.append('')  # Append an empty string if element is not found

        try:
            work_times.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__job-insight").text)
        except:
            work_times.append('')  # Append an empty string if element is not found

        time.sleep(2)
        
        # Scraping the job description
    job_description = driver.find_elements(By.CLASS_NAME,'jobs-description__content')
    for description in job_description:
        job_text = description.find_element(By.CLASS_NAME,"jobs-box__html-content").get_attribute('innerHTML')
        job_desc.append(str(job_text))
        time.sleep(2)  
    
    print(f'Scraping the Job Offer {i+1} DONE.')


C:\Users\papan\AppData\Local\Temp\ipykernel_14616\3382638812.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Visiting the links and collecting information just started.
Scraping the Job Offer 1 DONE.
Scraping the Job Offer 2 DONE.


In [138]:
import pandas as pd
Jobs_df = pd.DataFrame({'Title':job_titles,'Company':company_names,'Location':company_locations,'Work Methods':work_methods,'Post Date':post_dates,'Work Times':work_times,'Job Description':job_desc})

In [139]:
Jobs_df

,Title,Company,Location,Work Methods,Post Date,Work Times,Job Description
0,Χειριστής μηχανών παραγωγής και υπεύθυνος βάρδ...,PALAMIDIS SA,"Athens, Attiki, Greece",On-site,3 weeks ago,Full-time,"\n <h2 class=""text-heading-large mb4"">\..."
1,IT BUSINESS ANALYST,Swatch Group,"Athens, Attiki, Greece",,1 week ago,Full-time · Associate,"\n <h2 class=""text-heading-large mb4"">\..."
